In [19]:
import numpy as np

In [20]:
from Actfunc import*
from init import*
from Loss import*

In [21]:
Relu(np.array([-2,0,3]))

array([0., 0., 3.])

In [22]:
class Layer:

    def __init__(self,dim_in,dim_out,init_fn,act_fn=Relu):
        self.weights=init_fn(dim_in,dim_out)
        self.bias=np.reshape(init_fn(1,dim_out),-1)
        self.act_fn=act_fn


    def fw(self,input):
        output=self.weights@input + self.bias
        Z=output
        if self.act_fn:
            output=self.act_fn(Z)
        return output,Z

        

    def gradient_last(self,onehot,A,Z):
        S=softmax(Z[-1])
        delta=S-onehot
        d_WL=np.outer(delta,A[-2])
        d_BL=delta
        return d_WL, d_BL



            
            




In [113]:
class FFNN:

    def __init__(self,input_size,hidden_sizes,output_size, init_fn,act_fn):
        self.layers=[]
        self.layers.append(Layer(dim_in=input_size,dim_out=hidden_sizes[0],init_fn=init_fn,act_fn=act_fn))
        for i in range(1,len(hidden_sizes)):
            self.layers.append(Layer(dim_in=hidden_sizes[i-1],dim_out=hidden_sizes[i],init_fn=init_fn,act_fn=act_fn))
        self.layers.append(Layer(dim_in=hidden_sizes[-1],dim_out=output_size,init_fn=init_fn,act_fn=False))

    

    def forward(self,input):

        A=[]
        Z=[]
        output=input
        for layer in self.layers:
            a,z=layer.fw(output)
            A.append(a)
            Z.append(z)
            output=a
        return output,A,Z
    
    def full_gradient(self,A,Z,onehot,input):
        gradients_w=[]
        gradients_b=[]
        dLi_dW,dLi_dB=self.layers[-1].gradient_last(onehot,A,Z)
        dLi_df=dLi_dB
        gradients_w.insert(0,dLi_dW)
        gradients_b.insert(0,dLi_dB)
        for i in range(len(self.layers)-2,0,-1):
            dLi_df=((self.layers[i+1].weights.T@dLi_df).T*(Z[i]>0)).T
            dLi_dB=dLi_df
            dLi_dW=np.outer(dLi_df,A[i-1])
            gradients_w.insert(0,dLi_dW)
            gradients_b.insert(0,dLi_dB)
        dLi_df=((self.layers[1].weights.T@dLi_df).T*(Z[0]>0)).T
        dLi_dB=dLi_df
        dLi_dW=np.outer(dLi_df,input)
        gradients_w.insert(0,dLi_dW)
        gradients_b.insert(0,dLi_dB)
        return gradients_w,gradients_b

    







In [125]:
test=FFNN(input_size=3,hidden_sizes=[2,20,17,8],output_size=5,init_fn=he_normal,act_fn=Relu)
test_input=np.array([14,5,6])
test_onehot=[0,0,0,0,1]
test_output,A,Z=test.forward(test_input)

In [126]:
gradient_w,gradient_b=test.full_gradient(A,Z,test_onehot,test_input)

In [132]:
gradient_w[4]

array([[ 0.07111409,  0.06034104,  0.09774324,  0.        ,  0.1543687 ,
         0.12235324,  0.06628195,  0.22509496],
       [ 0.02166731,  0.01838493,  0.02978078,  0.        ,  0.04703364,
         0.03727905,  0.02019503,  0.06858278],
       [ 0.05609115,  0.04759392,  0.07709487,  0.        ,  0.12175813,
         0.09650598,  0.05227981,  0.17754339],
       [ 0.24886158,  0.21116162,  0.34204948,  0.        ,  0.54020854,
         0.42817144,  0.23195165,  0.78771293],
       [-0.39773413, -0.33748151, -0.54666836, -0.        , -0.86336901,
        -0.68430972, -0.37070843, -1.25893406]])

In [122]:
A[0]

array([5.02149641, 0.        ])